# Preprocessing

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
df1 = pd.read_csv('df1.csv', header=0)
df2 = pd.read_csv('df2.csv', header=0)
df3 = pd.read_csv('yellow_tripdata_2016-03.csv')
df4 = pd.read_csv('yellow_tripdata_2016-04.csv')

In [5]:
df2.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-02-25 17:00:00,2016-02-25 17:00:00,2,0.70,-73.947250,40.763771,1,N,-73.992012,40.735390,2,5.0,0.0,0.5,0.0,0.0,0.3,5.8
1,2,2016-02-25 23:00:00,2016-02-25 23:00:00,2,5.52,-73.983017,40.750992,1,N,-73.988586,40.758839,2,20.0,0.5,0.5,0.0,0.0,0.3,21.3
2,2,2016-02-01 00:00:00,2016-02-01 00:00:00,6,1.99,-73.992340,40.758202,1,N,-73.964355,40.757977,1,9.5,0.5,0.5,0.7,0.0,0.3,11.5
3,1,2016-02-01 00:00:00,2016-02-01 00:00:00,1,1.50,-73.981453,40.749722,1,N,-73.982323,40.763985,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8
4,2,2016-02-01 00:00:00,2016-02-01 00:00:00,1,5.60,-74.000603,40.729755,1,N,-73.951324,40.669834,1,20.0,0.5,0.5,4.0,0.0,0.3,25.3


In [4]:
def preprocessing(df): 
    df = df[df['tpep_dropoff_datetime'] != df['tpep_pickup_datetime']]
    df = df[df['passenger_count'] != 0]
    df = df[df['trip_distance'] != 0]
    df = df[(df['pickup_longitude'] >= -74.25559) & (df['pickup_longitude'] <= -73.70001) 
           & (df['pickup_latitude'] >= 40.49612) & (df['pickup_latitude'] <= 40.91553)]
    df['tpep_pickup_datetime'] =  pd.to_datetime(df['tpep_pickup_datetime'], format='%Y%m%d %H:%M:%S.%f')
    df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].map(lambda x: x.replace(second=0))
    df['tpep_pickup_datetime'] = df['tpep_pickup_datetime'].map(lambda x: x.replace(minute=0))
    df['tpep_dropoff_datetime'] =  pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y%m%d %H:%M:%S.%f')
    df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].map(lambda x: x.replace(second=0))
    df['tpep_dropoff_datetime'] = df['tpep_dropoff_datetime'].map(lambda x: x.replace(minute=0))
    return df

In [5]:
df1 = preprocessing(df1)
df2 = preprocessing(df2)
df3 = preprocessing(df3)
df4 = preprocessing(df4)

In [16]:
df1.to_csv('df1.csv', index=False)
df2.to_csv('df2.csv', index=False)
df3.to_csv('df3.csv', index=False)
df4.to_csv('df4.csv', index=False)

In [ ]:
%%time
for i in range(31*24 + 29*24 + 31*24 + 30*24):
    cur_datetime = df['tpep_pickup_datetime'].values[0] + np.timedelta64(i, 'h')
    df_cut_datetime = df[df['tpep_pickup_datetime'] == cur_datetime]
    ret = stats.binned_statistic_2d( df_cut_datetime['pickup_longitude'],  df_cut_datetime['pickup_latitude'],
                      None,  statistic='count',
                      bins=[np.linspace(-74.25559, -73.70001, 51) , np.linspace(40.49612, 40.91553, 51)])
    trips_by_regions[(str(cur_datetime)[:10] + '(hour: {})'.format(i%24))] = ret.statistic.reshape(2500)
    if i%24==0 and i!=0:
        print('day {}'.format(i/24))